### Jul AdventKalender D22

https://adventofcode.com/2023/day/22

#### Part 1 

In [1]:
blocks = []
with open("data/input22.txt","r") as file:
    for line in file.readlines():
        s,e = line.strip().split('~')
        blocks.append([[int(i) for i in s.split(',')],[int(i) for i in e.split(',')]])
blocks = sorted(blocks, key = lambda x: x[0][2]) # by z in the start pos

In [2]:
import copy

def if_disjoint(check_block, block):
    return (check_block[1][0] < block[0][0] or block[1][0] < check_block[0][0]) or (check_block[1][1] < block[0][1] or block[1][1] < check_block[0][1])

def can_fall(block, previous_fall):
    if len(previous_fall)==0:
        return 1
    fall_h = -1
    # check each level below
    for i in range(block[0][2]-1, 0, -1):
        if str(i) in previous_fall:
            check_blocks = previous_fall[str(i)]
            is_any_block = False
            for check_block in check_blocks:
                if not if_disjoint(check_block, block):
                    is_any_block = True
                    break
            if is_any_block == False:
                fall_h = i
                continue
            else:
                break
        else:
            fall_h = i
            continue
    return fall_h

def fall(blocks):
    res = []
    level_blocks = {}
    for block in blocks:
        hight = block[1][2] - block[0][2]
        fall_h = can_fall(block, level_blocks)
        block_new = copy.deepcopy(block)
        if fall_h > 0: # can fall
            block_new[0][2] = fall_h
            block_new[1][2] = block_new[0][2] + hight
        for i in range(block_new[0][2], block_new[1][2]+1):
            level_blocks[str(i)] = level_blocks.get(str(i), []) + [block_new]
        #print("fall==", block_new)
        res.append(block_new)
    return res, level_blocks

In [3]:
def disintegrate_no_fall(tar_blocks, level_blocks):
    res = set()
    block_supports_by = {}
    for cur_level, cur_blocks in level_blocks.items():
        if cur_level == list(level_blocks.keys())[-1]:
            for v in cur_blocks:
                res.add(str(v))
        up_blocks = level_blocks.get(str(int(cur_level)+1))
        if up_blocks:
            for cur_block in cur_blocks:
                if cur_block[1][2]!=int(cur_level):
                    continue
                n_disjoint = 0
                for up_block in up_blocks:
                    if up_block[0][2]!=int(cur_level)+1:
                        n_disjoint += 1
                        continue
                    if not if_disjoint(cur_block, up_block):
                        block_supports_by[str(up_block)] = block_supports_by.get(str(up_block), []) + [str(cur_block)]
                    else:
                        n_disjoint += 1
                if n_disjoint == len(up_blocks):
                    #print(cur_block)
                    res.add(str(cur_block))
        for k, vs in block_supports_by.items():
            if len(vs) > 1:
                for v in vs:
                    res.add(v)
        for k, vs in block_supports_by.items():
            if len(vs) <= 1:
                res.discard(vs[0])
    return res, block_supports_by

fall_blocks, level_blocks = fall(blocks)
safe_disintegrate_blocks, block_supports_by = disintegrate_no_fall(fall_blocks, level_blocks)
len(safe_disintegrate_blocks)

443

#### Part 2

In [4]:
# next checks: check next time, all above being supported by the tar_blocks
# for the ones that being supported but higher (top z) than the next checking level, 
# save for when the next checking level is the same => save_checks
def n_chain(tar_blocks, block_supports_by, level, n=0, save_checks=[]):
    next_checks = []
    for up_block, supprt_blocks in block_supports_by.items():
        if set(supprt_blocks).issubset(tar_blocks): # chain of effects
            if int(up_block.replace('[','').replace(']','').split(',')[5]) <= level+1:
                next_checks.append(up_block)
            else:
                save_checks.append(up_block)
    for i in range(len(save_checks)-1,-1,-1):
        save_check = save_checks[i]
        if int(save_check.replace('[','').replace(']','').split(',')[5]) <= level+1:
            next_checks.append(save_check)
            del save_checks[i]
    if next_checks or save_checks: 
        return n_chain(next_checks, block_supports_by, level+1, n+len(next_checks), save_checks)
    return n

total = 0
for level, blocks in level_blocks.items():
    for block in blocks:
        # only check when the top z is the save level as current level
        if block[1][2] == int(level) and not set(str(block)).issubset(safe_disintegrate_blocks):
            n = n_chain([str(block)], block_supports_by, int(level))
            #print(block, n)
            total += n
            
total

69915